# Implementing my own graph class and Page ranking

## Warmup using networkx


In [42]:
# AUTOLAB_IGNORE_START
import networkx as nx
Gr = nx.read_edgelist("wikipedia_small.graph",create_using=nx.DiGraph())
Gr.edges()
nx.shortest_path_length(G, source = "4368")

## My own Graph class

## Implement `add_edges()` call

To begin, implement the function `add_edges()`.  This will modify the `self.edges` variable to add all edges passed as tuples in `edges_list`.  Note that `self.edges` should be represented as an "adjacency dictionary", so that for every node `i` in the graph

    self.edges[i]
    
indicators another dictionary of nodes that `i` is connected to (with the direction pointing from `i` to the other node).  For instance, if `self.edges[i][j]` is a valid entry this means that there is an edge from node `i` to node `j` (the value of this entry doesn't matter, so we could technically use a dictionary of sets, but we use a dictionary of dictionaries to keep things a little bit more uniform and to allow for potential extensions e.g. to weighted graphs).

For instance, the following code populates a simple graph like the following:

    A -> B -> C
    |    ^
    v    |
    D -> E
    
Note that all vertices must exist in the dictionary. If a vertex has no outgoing edges, then it should still have an entry pointing to an empty dictionary. 

## Djisktra's algorithm

Next, implement Djisktra's single-source shortest path algorithm (with the simple case where the distance along any edge is assumed to be one).  We can refer to the Wikipedia page on Djikstra's algorithm for reference (https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm), but the basic idea of the algorithm is to keep a priority queue of the current known distances to a node.  We continually pop off the smallest element `i` in the queue, and then update all its successor nodes to have a distance of 1 + distance[i].

For the priority queue, we should use the heapdict library (https://github.com/DanielStutzbach/heapdict) that is included above (and included with Anaconda).  We can update the priority of a heapdict element and pop off the smallest element using the syntax

    d = heapdict({"a":5, "b":6})
    d["b"] = 4
    d.popitem() # -> ("b", 4)
    ...
    
    
The function should return a list both of the shortest distances and the previous nodes in the shortest path from the source node to this node. 

* The distance for unreachable nodes should be inf
* The path for the source node should be None


## Adjacency matrix representation

Implement the adjacency matrix method of the Graph class.  This returns a matrix representing the adjacency of the graph (in scipy COO sparse format), as well as a list of nodes that indicate how the indices in this graph relate to the nodes in the network.

IMPORTANT: in order to complete this question in a manner that works on the Wikipedia, we will _need_ to implement this function natively as a sparse matrix (i.e., we cannot construct a dense matrix and then convert that to a sparse matrix, but need to directly use the `sp.coo_matrix()` constructor).  The Wikipedia graph is is 24K x 24K nodes, which (assuming 8 bytes per entry, would take up 4GB of memory.  While it's not impossible to do things this way at this scale (it quickly becomes infeasible for graphs that are even slightly larger), it's a very bad idea, and just allocating this much memory will take too long.

For example, in our graph above we have the following:

## PageRank algorithm

Finally, implement the PageRank algorithm using the adjacency matrix representation that we built in the previous question.  Reference on the PageRank algorithm is here: https://en.wikipedia.org/wiki/PageRank.  We should specifically use the approach described in the "Power Method" section on this page, which we also discussed in class.  This involves forming some initial uniform probability vector $x$, and repeatly multiplying it by the matrices:
\begin{equation}
x := (d P + ((1-d)/n) E)x
\end{equation}
where $P$ is a transition matrix (the $A$ matrix normalized so that all columns have sum 1), $E$ is the matrix of all ones, and $d$ is the damping factor.

NOTES: Recall that from the definition of PageRank, when we reach a "sink" node (a node with no outgoing edges), we randomly hope to any other node in the network, so that columns of $P$ that have no outgoing edges are set to the uniform distribution.  To be efficient, we'll also want to avoid explicitly forming the $E$ matrix, and should instead use the fact that $E = 11^T$ where $1$ denotes a vector of all ones.  Use the fact that we can reorder matrix multiplication if associative (i.e., the fact the $A(BC)$ = $(AB)C$) to make this operation as fast as possible.

Our function should return a dictionary of nodes and their corresponding page rank.  For example, in our graph above, we have the following reults:

In [1]:
import numpy as np
import scipy.sparse as sp
import heapdict

class Graph:
    def __init__(self):
        """ Initialize with an empty edge dictionary. """
        self.edges = {}
    
    def add_edges(self, edges_list):
        """ Add a list of edges to the network. Use 1.0 to indiciate the presence of an edge. 
        
        Args:
            edges_list: list of (a,b) tuples, where a->b is an edge to add
        """
        for thisTuple in edges_list:
            if thisTuple[0] in self.edges:
                if thisTuple[1] not in self.edges[thisTuple[0]]:
                    self.edges[thisTuple[0]][thisTuple[1]] = 1.0
            else:
                self.edges[thisTuple[0]] = {}
                self.edges[thisTuple[0]][thisTuple[1]] = 1.0
            if thisTuple[1] not in self.edges:
                self.edges[thisTuple[1]] = {}
        
        pass
        
    def shortest_path(self, source):
        """ Compute the single-source shorting path.
        
        This function uses Djikstra's algorithm to compute the distance from 
        source to all other nodes in the network.
        
        Args:
            source: node index for the source
            
        Returns: tuple: dist, path
            dist: dictionary of node:distance values for each node in the graph, 
                  where distance denotes the shortest path distance from source
            path: dictionary of node:prev_node values, where prev_node indicates
                  the previous node on the path from source to node
        """
        dist_map = {}
        path_map = {}
        
        initial_node = source
        dist_map[source] = 0
        path_map[source] = None
        
        for key in self.edges:
            if key not in dist_map:
                dist_map[key] = float('inf')
                path_map[key] = None
                
        d = heapdict.heapdict(dist_map)
        
        while(len(d) > 0):
            current = d.popitem()

            if current[1] == float('inf'):
                continue

            for key in self.edges[current[0]]:
                if key in d:
                    dist_map[key] = min(dist_map[key], dist_map[current[0]]+1)
                    if path_map[key] == None:
                        path_map[key] = current[0]    
                    d[key] = dist_map[key]

        return (dist_map, path_map) 
        pass
    
        
    def adjacency_matrix(self):
        """ Compute an adjacency matrix form of the graph.  
        
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjaency matrix
               for the graph (i.e., A[j,i] = 1 iff there is an edge i->j)
               NOTE: be sure you have this ordering correct!
            nodes: a list of nodes indicating the node key corresponding to each
                   index of the A matrix
        """
        index_dict = {}
        index_ctr = 0
        nList = []
        for key in self.edges:
            index_dict[key] = index_ctr
            index_ctr += 1
            nList.append(key)
        
        row = []
        col = []
        data = []
        for pkey in self.edges:
            if len(self.edges[pkey]) > 0:
                for ckey in self.edges[pkey]:
                    col.append(index_dict[pkey])
                    row.append(index_dict[ckey])
                    data.append(1)
        
        A = sp.coo_matrix((data, (row, col)),shape = (len(self.edges), len(self.edges)))

        return (A, nList)

    
    def pagerank(self, d=0.85, iters=100):
        """ Compute the PageRank score for each node in the network.
        
        Compute PageRank scores using the power method.
        
        Args:
            d: 1 - random restart factor
            iters: maximum number of iterations of power method
            
        Returns: dict ranks
            ranks: a dictionary of node:importance score, for each node in the
                   network (larger score means higher rank)
        
        """
        rank_dict = {}
        with np.errstate(divide='ignore'):
            M,nList = self.adjacency_matrix()
            M_csr = M.tocsr()
            n,_ = M_csr.shape

            rank_vect = np.ones((n,1))/n
            out_deg = M_csr.sum(axis=0)
            out_deg_vect = out_deg.T / d

            while iters > 0:
                new_rank_vect = M_csr.dot((rank_vect/out_deg_vect))
                new_rank_vect += (1-new_rank_vect.sum())/n
                rank_vect = new_rank_vect
                iters -= 1
            rank_list = []
            for item in rank_vect.tolist():
                rank_list.append(item[0])
            
            return dict(zip(nList, rank_list))
            

In [290]:
G = Graph()
# G.add_edges([("A","B"), ("B","A"), ("C", "B"), ("C","A")])
G.add_edges([("A","B"), ("B","C"), ("A","D"), ("D", "E"), ("E", "B")])
print(G.edges)
# G.shortest_path("C")


{'A': {'B': 1.0, 'D': 1.0}, 'B': {'C': 1.0}, 'C': {}, 'D': {'E': 1.0}, 'E': {'B': 1.0}}


In [292]:
# AUTOLAB_IGNORE_START
# print(G.shortest_path('1753'))
print(G.shortest_path("B"))
print(G.shortest_path("C"))
print(G.shortest_path("D"))
# print(G.shortest_path("E"))
# AUTOLAB_IGNORE_STOP

In [293]:
# AUTOLAB_IGNORE_START
A, nlist = G.adjacency_matrix()
print(A)
print(A.tocsr())
print(nlist)
# AUTOLAB_IGNORE_STOP

  (1, 0)	1
  (3, 0)	1
  (2, 1)	1
  (4, 3)	1
  (1, 4)	1
  (1, 0)	1
  (1, 4)	1
  (2, 1)	1
  (3, 0)	1
  (4, 3)	1
['A', 'B', 'C', 'D', 'E']


In [294]:
# AUTOLAB_IGNORE_START
G.pagerank()
# AUTOLAB_IGNORE_STOP

{'A': 0.0851086238706817,
 'B': 0.2812467668696596,
 'C': 0.3241683757098924,
 'D': 0.12127978901572142,
 'E': 0.1881964445340449}



    United_States 0.00275188705264
    2007 0.00244339053989
    2008 0.00217342514773
    Wikimedia_Commons 0.00172604200122
    United_Kingdom 0.00159998851013
    2006 0.00154844911674
    France 0.00144958994072
    Wiktionary 0.00126753476354
    Canada 0.00109896195215
    World_War_II 0.00104913079624
    2005 0.00104633024568
    List_of_African_films 0.00100713870383
    Germany 0.000956262192248
    Europe 0.000937690025073
    English_language 0.000908144359626
    Geographic_coordinate_system 0.000891711151403
    Latin 0.000888662228804
    Australia 0.000879854710005
    India 0.000787625093175
    Japan 0.000784815389935

